In [3]:
# ===========================
# COMP0078 Coursework 2 Q1.1
# Douglas Chiang
# 15055142
# ===========================
cd /content/drive/MyDrive/Colab Notebooks/COMP0078/CW2/Q1

/content/drive/MyDrive/Colab Notebooks/COMP0078/CW2/Q1


In [4]:
ls

 Assignment_2_Question_1a_v3.ipynb     dtrain123.dat     zipcombo.dat
 Assignment_2_Question_1bcd_v1.ipynb   Plots/
 dtest123.dat                         'Q1 Trial.ipynb'


Libraries:

In [5]:
import time
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt

Functions for data

In [6]:
def load_dat(fname, Full_Data = False):
    dat = pd.read_csv(fname, sep='\s+', header=None).to_numpy()
    if Full_Data == False:
        y = dat[:, 0].astype(np.int) - 1  # we are zero-based indexing
    else:
        y = dat[:, 0].astype(np.int)
    
    X = dat[:, 1:].astype(np.float64)
    return X, y

def Plot_Accuracy_SV(Ker_para, run, n_epochs, Train_accuracy_All, Test_accuracy_All):
    fig1, ax1 = plt.subplots()

    ax1.plot(range(1, n_epochs), Train_accuracy_All, color='red', label='Training accuracy')
    ax1.plot(range(1, n_epochs), Test_accuracy_All, color='blue', label='Testing accuracy')

    plt.xlabel('Epoch(s)')
    leg = ax1.legend(bbox_to_anchor=(1.5, 1.02), loc='upper right', frameon=False)
    plt.savefig(f"/content/drive/MyDrive/Colab Notebooks/COMP0078/CW2/Q1/Plots/Q1a/d_{Ker_para}_run_{run}_Accuracy_SV.png")

Kernel Calculation

In [7]:
class KernelFunctor:
    """
    Calculate the Kernel values and put them in a matrix
    """
    def __init__(self, Ker_Meth="Poly", Ker_para=2):
        self.Ker_Meth = Ker_Meth
        self.Ker_para = Ker_para
        #self.X_idx = 
    
    def kernel_fn(self, X):
        if self.Ker_Meth == "Poly":
            return (X.dot(X.T))**self.Ker_para
        elif self.Ker_Meth == "Gau":
            pass# return np.exp(-self.Ker_para*np.linalg.norm(p_vec - q_vec)**2)
        else:
            assert False, "Kernel Method Undefined"

K - Class Classifier: One vs Rest Online Perceptron

In [8]:
class OvR_OnlinePerceptron:
    """
    One-vs-Rest
    """
    def __init__(self, alphas, kernel_matrix, n_classes=3):
        self.alphas              = alphas
        self.kernel_matrix       = kernel_matrix
        self.alphas              = alphas
        self.n_classes           = n_classes

    def fit(self, ran_train_sample_idx, X, y):
        n_samples, n_dim = X.shape

        # record down data
        self.n_dim                = n_dim
        self.n_samples            = range(n_samples)
        self.ran_train_sample_idx = ran_train_sample_idx
        # ------------------------------------------------------
        for i in self.n_samples:
            cls_vals           = self._get_classifier_vals(self.ran_train_sample_idx[i])
            preds              = self.sign(cls_vals)
            truth              = self._label_to_custom_onehot(y[i])
            self.alphas[:, i] -= np.where(np.multiply(cls_vals, truth) <= 0, preds, 0)

        return self.alphas

    def _get_classifier_vals(self, sample_idx):
        """
        This is the classifier:
        .. math::
            w(x) = \sum_i \alpha_i K(x_i, x)
        Args:
            x (np.ndarray): the input feature you would like to classify
        Return:
            An array of size (n_classes,). Each value represents the inner product
            sum between support vectors of a classifier and the incoming feature.
        """
        ret           = np.zeros((self.n_classes,))
        kernel_values = self.kernel_matrix[self.ran_train_sample_idx, sample_idx]
        ret          += self.alphas.dot(kernel_values)
        assert ret.shape == (self.n_classes,)
        return ret

    def predict(self, sample_idx):
        cls_vals = self._get_classifier_vals(sample_idx)
        return np.argmax(cls_vals)

    @staticmethod
    def sign(val):
        ret = np.where(val <= 0.0, -1, 1)
        return ret

    def _label_to_custom_onehot(self, label: int):
        """
        Similar to one-hot encoding, we mark the truth one as 1, otherwise -1
        Example:
        >>> svm = MulticlassSVM(n_classes=5)
        >>> svm._label_to_custom_onehot(2)
        [-1, -1, 1, -1, -1]
        >>> svm._label_to_custom_onehot(4)
        [-1, -1, -1, -1, 1]
        """
        ret = np.full((self.n_classes,), -1)
        ret[label] = 1
        return ret

In [9]:
# Trial data set:
# X_train, y_train = load_dat("dtrain123.dat", Full_Data = False)
# X_test, y_test   = load_dat("dtest123.dat", Full_Data = False)
# X_dataset = np.vstack((X_train, X_test))
# Y_dataset = np.hstack((y_train, y_test))

# Full data set:
X_dataset, Y_dataset = load_dat("zipcombo.dat", Full_Data = True)

In [10]:
# K-Class Classifier Parameters:
Ker_Meth  = "Poly"
n_classes = 10

# Question 1a):
Training_Error_Mean  = []
Training_Error_Stdev = []
Testing_Error_Mean   = []
Testing_Error_Stdev  = []

# ---------------Training--------------------------
# Question 1a):
# Loop for d from 1 to 7
for Ker_para in range(1,8):
    print("d = {}".format(Ker_para))

    # Question 1a:
    Training_Error_per_run = []
    Testing_Error_per_run  = []

    print("Precalculate Kernel...")
    KF  = KernelFunctor(Ker_Meth, Ker_para)
    tic = time.perf_counter()
    KN  = KF.kernel_fn(X_dataset) # Kernel matrix
    toc = time.perf_counter()
    print(f"**Kernel calculation finished in {toc - tic:0.4f} seconds")

    # 20 runs:
    for run in range(1,21):
        print("Run: {}".format(run))

        # Index Shuffle for fitting:
        ran_sample_idx       = np.random.permutation(len(Y_dataset))
        split_pt             = round(len(ran_sample_idx)*0.8)

        # Index for Debug:
        # ran_sample_idx       = range(len(Y_dataset))
        # split_pt             = round(len(ran_sample_idx)*0.41911)

        ran_train_sample_idx = ran_sample_idx[:split_pt]
        ran_test_sample_idx  = ran_sample_idx[split_pt:len(ran_sample_idx)]

        # Data sets:
        X_train              = X_dataset[ran_train_sample_idx]
        Y_train              = Y_dataset[ran_train_sample_idx]
        X_test               = X_dataset[ran_test_sample_idx]
        Y_test               = Y_dataset[ran_test_sample_idx]

        nsamples             = len(ran_train_sample_idx)
        n_test_samples       = len(ran_test_sample_idx)

        # Initialize:
        alphas               = np.zeros((n_classes, nsamples))
        # alphas               = np.zeros((n_classes, nsamples + n_test_samples))

        # Start Model:
        ovrop = OvR_OnlinePerceptron(alphas, KN, n_classes)

        # Results memory:
        Train_correct_All    = []
        Train_accuracy_All   = []
        Train_mistakes_All   = []
        Test_correct_All     = []
        Test_accuracy_All    = []
        Test_mistakes_All    = []

        n_epochs = 20
        for epoch in range(1, n_epochs + 1):
            tic    = time.time()
            alphas = ovrop.fit(ran_train_sample_idx, X_train, Y_train)
            toc    = time.time()

        # ---------------Training Accuracy-----------------
            Train_correct = 0

            # kernel_vals   = KN[:, ran_train_sample_idx]
            kernel_vals   = KN[ran_train_sample_idx]
            kernel_vals   = kernel_vals[:, ran_train_sample_idx]
            pred          = np.array(list(map(np.argmax, zip(*(alphas.dot(kernel_vals))))))
            Train_correct = len(np.where((Y_train - pred) == 0)[0])
            
            # Record results:
            Train_correct_All.append(Train_correct)

            Train_accuracy = Train_correct / nsamples
            Train_accuracy_All.append(Train_accuracy)

            Train_mistakes = nsamples - Train_correct
            Train_mistakes_All.append(Train_mistakes)

            # print("\tTraining correctness = ", Train_correct)
            # print("\tTraining accuracy    = ", Train_accuracy)
            # print("\tTraining mistake(s)  = ", Train_mistakes, "\n")

        # ---------------Testing Accuracy-------------------
            # print(">>Testing:")
            Test_correct = 0

            # kernel_vals  = KN[:, ran_test_sample_idx]
            kernel_vals  = KN[ran_train_sample_idx]
            kernel_vals  = kernel_vals[:, ran_test_sample_idx]
            pred         = np.array(list(map(np.argmax, zip(*(alphas.dot(kernel_vals))))))
            Test_correct = len(np.where((Y_test - pred) == 0)[0])

            # Record results:
            Test_correct_All.append(Test_correct)

            Test_accuracy = Test_correct / n_test_samples
            Test_accuracy_All.append(Test_accuracy)

            Test_mistakes = n_test_samples - Test_correct
            Test_mistakes_All.append(Test_mistakes)

            # print("\tTesting correctness  = ", Test_correct)
            # print("\tTesting accuracy     = ", Test_accuracy)
            # print("\tTesting mistake(s)   = ", Test_mistakes, "\n")
            print(f"**Training: Epoch {epoch} finished in {toc - tic:0.4f} seconds with Train accuracy {Train_accuracy:0.4f} and Test accuracy {Test_accuracy:0.4f}.")
            epoch += 1

        # Question 1a): Save error rate:
        Training_Error_per_run.append(Train_mistakes/nsamples)
        Testing_Error_per_run.append(Test_mistakes/n_test_samples)

        Plot_Accuracy_SV(Ker_para, run, n_epochs + 1, Train_accuracy_All, Test_accuracy_All)

    # Question 1a): Error mean and std:
    Training_Error_Mean.append(np.mean(Training_Error_per_run))
    Training_Error_Stdev.append(np.std(Training_Error_per_run))
    Testing_Error_Mean.append(np.mean(Testing_Error_per_run))
    Testing_Error_Stdev.append(np.std(Testing_Error_per_run))

print("Results:")
print(f"Training Error Mean: {Training_Error_Mean}")
print(f"Training Error STD:  {Training_Error_Stdev}")
print(f"Testing  Error Mean: {Testing_Error_Mean}")
print(f"Testing  Error STD:  {Testing_Error_Stdev}")

Output hidden; open in https://colab.research.google.com to view.